In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.stats import describe
from sklearn.metrics import mean_absolute_error

#leer archivo csv
archivo_csv = r"C:\Users\hctrl\OneDrive\Tesis\Experimento\Temperatura Máxima\TempMax.csv"
datos = pd.read_csv(archivo_csv)

# Configurar la opción para mostrar todas las filas
pd.set_option('display.max_rows', None)

print(datos)

        ID_CIDERH  TEMP_MAX_RBE  Anio  Mes  Dia
0      M_017_A_II     16.704683  1993    1    1
1      M_017_A_II     16.886888  1993    1    2
2      M_017_A_II     16.676035  1993    1    3
3      M_017_A_II     16.824541  1993    1    4
4      M_017_A_II     15.862032  1993    1    5
5      M_017_A_II     14.994508  1993    1    6
6      M_017_A_II     14.162866  1993    1    7
7      M_017_A_II     12.723688  1993    1    8
8      M_017_A_II     12.663499  1993    1    9
9      M_017_A_II     13.711763  1993    1   10
10     M_017_A_II     12.748041  1993    1   11
11     M_017_A_II     13.041406  1993    1   12
12     M_017_A_II     12.755754  1993    1   13
13     M_017_A_II     14.052637  1993    1   14
14     M_017_A_II     14.090929  1993    1   15
15     M_017_A_II     13.058794  1993    1   16
16     M_017_A_II     15.723825  1993    1   17
17     M_017_A_II     15.873804  1993    1   18
18     M_017_A_II     15.188989  1993    1   19
19     M_017_A_II     13.165666  1993   

In [3]:
# Cargar el archivo CSV existente
df = pd.read_csv(r"C:\Users\hctrl\OneDrive\Tesis\Experimento\Temperatura Máxima\TempMax.csv")

# Obtener fechas únicas para la estación M_023_A_II
fechas_estacion = pd.to_datetime(df[df['ID_CIDERH'] == 'M_023_A_II']['Anio'].astype(str) + '-' +
                                 df[df['ID_CIDERH'] == 'M_023_A_II']['Mes'].astype(str) + '-' +
                                 df[df['ID_CIDERH'] == 'M_023_A_II']['Dia'].astype(str))

# Establecer la cantidad deseada de fechas adicionales a generar
cantidad_fechas_adicionales = 15

# DataFrame acumulativo inicialmente igual al DataFrame original
df_acumulativo = df.copy()

# Crear un directorio para almacenar los archivos CSV y gráficos
output_directory = 'Resultados'
os.makedirs(output_directory, exist_ok=True)

# Contador de fechas generadas con valores nulos
fechas_nulas_generadas = 0

# Iterar para generar fechas y acumular los cambios
while fechas_nulas_generadas < cantidad_fechas_adicionales:
    # Elegir aleatoriamente una fecha dentro del rango de fechas disponibles para la estación M_015_A_II
    fecha_aleatoria = pd.Timestamp(np.random.choice(fechas_estacion))

    # Modificar el DataFrame acumulativo estableciendo TEMP_MAX_RBE en nulo para la estación M_015_A_II y la fecha aleatoria
    ventana_temporal = 7
    fecha_inicio = max(fecha_aleatoria - pd.Timedelta(days=ventana_temporal), fechas_estacion.min())
    fecha_fin = min(fecha_aleatoria + pd.Timedelta(days=ventana_temporal), fechas_estacion.max())

    mask = (df_acumulativo['ID_CIDERH'] == 'M_023_A_II') & (fechas_estacion >= fecha_inicio) & (
                fechas_estacion <= fecha_fin)

    # Verificar si hay índices antes de modificar
    if not df_acumulativo.loc[mask].empty:
        indice_elegido = np.random.choice(df_acumulativo.loc[mask].index)
        df_acumulativo.loc[indice_elegido, 'TEMP_MAX_RBE'] = np.nan
        fechas_nulas_generadas += 1

    # Crear el nombre del archivo CSV
    csv_filename = f'TempMax_con_vacios_{fechas_nulas_generadas}.csv'

    # Crear el directorio para cada archivo CSV
    csv_directory = os.path.join(output_directory, csv_filename.replace('.csv', ''))
    os.makedirs(csv_directory, exist_ok=True)

    # Guardar el DataFrame acumulativo como un nuevo archivo CSV en el directorio correspondiente
    csv_path = os.path.join(csv_directory, csv_filename)
    df_acumulativo.to_csv(csv_path, index=False)

    # Configurar la opción para mostrar todas las filas
    pd.set_option('display.max_rows', None)

    # Leer el archivo CSV
    df = pd.read_csv(csv_path)

    # Filtrar datos para la estación ID_CIDERH M_023_A_II
    df_estacion = df[df['ID_CIDERH'] == 'M_023_A_II']

    # Crear una columna de fecha combinando mes, día y año
    df_estacion['Fecha'] = pd.to_datetime(df_estacion['Anio'].astype(str) + '-' +
                                          df_estacion['Mes'].astype(str) + '-' +
                                          df_estacion['Dia'].astype(str))

    # Configurar el estilo del gráfico
    sns.set(style="whitegrid")
    plt.figure(figsize=(15, 6))

    # Crear el gráfico de líneas para la temperatura max
    plt.plot(df_estacion['Fecha'], df_estacion['TEMP_MAX_RBE'], label='Temperatura máxima', color='blue')

    # Configurar el gráfico
    plt.title(f'Comportamiento de la temperatura máxima para la estación M_023_A_II')
    plt.xlabel('Fecha')
    plt.ylabel('Temperatura Máxima (°C)')
    plt.xticks(rotation=45)
    plt.legend()

    # Guardar el gráfico en el directorio correspondiente
    plt.savefig(os.path.join(csv_directory, f'TEMP_MAX_RBE_Grafico_{fechas_nulas_generadas}.png'))
    plt.close()  # Cerrar el gráfico para evitar la superposición en la siguiente iteración

# Imprimir mensaje al finalizar el proceso
print(f'Se han generado y guardado {fechas_nulas_generadas} DataFrames acumulativos con nulos.')

C:\Users\hctrl\AppData\Local\Temp\ipykernel_18716\132067661.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estacion['Fecha'] = pd.to_datetime(df_estacion['Anio'].astype(str) + '-' +
C:\Users\hctrl\AppData\Local\Temp\ipykernel_18716\132067661.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estacion['Fecha'] = pd.to_datetime(df_estacion['Anio'].astype(str) + '-' +
C:\Users\hctrl\AppData\Local\Temp\ipykernel_18716\132067661.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy 

Se han generado y guardado 15 DataFrames acumulativos con nulos.


C:\Users\hctrl\AppData\Local\Temp\ipykernel_18716\132067661.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estacion['Fecha'] = pd.to_datetime(df_estacion['Anio'].astype(str) + '-' +


In [3]:
# Leer el archivo CSV
archivo_csv2 = r"C:\Users\hctrl\OneDrive\Tesis\Experimento\Temperatura Máxima\Vacios Aleatorios\Resultados\TempMax_con_vacios_15\TempMax_con_vacios_15.csv"
datos2 = pd.read_csv(archivo_csv2)

# Buscar valores nulos en la columna 'TEMP_MAX_RBE'
fechas_con_nulos = datos2[datos2['TEMP_MAX_RBE'].isnull()][['ID_CIDERH', 'TEMP_MAX_RBE', 'Anio', 'Mes', 'Dia']]

print("Fechas con valores nulos generados:")
print(fechas_con_nulos)

Fechas con valores nulos generados:
        ID_CIDERH  TEMP_MAX_RBE  Anio  Mes  Dia
12450  M_023_A_II           NaN  1993   10    6
12946  M_023_A_II           NaN  1995    2   14
12953  M_023_A_II           NaN  1995    2   21
13484  M_023_A_II           NaN  1998    6   17
13703  M_023_A_II           NaN  1999    5   13
13796  M_023_A_II           NaN  1999    8   14
14252  M_023_A_II           NaN  2000   11   24
14453  M_023_A_II           NaN  2001    7    7
16036  M_023_A_II           NaN  2005   11   14
16152  M_023_A_II           NaN  2006    3   10
16537  M_023_A_II           NaN  2007    3   30
16632  M_023_A_II           NaN  2007    7    3
16686  M_023_A_II           NaN  2007    8   26
17091  M_023_A_II           NaN  2008   10   22
17425  M_023_A_II           NaN  2009    9   24


In [5]:
# Cargar los 15 dataframes con valores faltantes
dataframes_con_vacios = [pd.read_csv(os.path.join('Resultados', f'TempMax_con_vacios_{i + 1}', f'TempMax_con_vacios_{i + 1}.csv')) for i in range(15)]

# Crear un diccionario que contenga las estaciones vecinas para cada estación
estaciones_vecinas = {
    'M_023_A_II': ['M_017_A_II', 'M_024_A_II', 'M_027_A_II'],
}

# Imputar utilizando la técnica de la razón normal para cada dataframe
dataframes_imputados = []

for df in dataframes_con_vacios:
    missing_values = df['TEMP_MAX_RBE'].isnull()

    if missing_values.any():
        for idx, row in df[missing_values].iterrows():
            estacion_actual = row['ID_CIDERH']
            estaciones_vecinas_actual = estaciones_vecinas.get(estacion_actual, [])

            # Filtrar el DataFrame para obtener observaciones de la estación actual y las vecinas
            estaciones_df = df[df['ID_CIDERH'].isin([estacion_actual] + estaciones_vecinas_actual)]
            observed_values = estaciones_df['TEMP_MAX_RBE'].dropna()

            mean_without_missing = observed_values.mean()
            std_without_missing = observed_values.std()

            # Generar valores imputados basados en una distribución normal estándar
            imputed_values = np.random.normal(loc=mean_without_missing, scale=std_without_missing, size=1)

            # Asignar los valores imputados a las filas con valores faltantes
            df.loc[idx, 'TEMP_MAX_RBE'] = imputed_values[0]

    dataframes_imputados.append(df)

# Guardar los 15 dataframes imputados
output_directory_imputados = 'Resultados_Imputacion_Razon_Normal'
os.makedirs(output_directory_imputados, exist_ok=True)

for i, df_imputado in enumerate(dataframes_imputados):
    csv_filename_imputado = f'TempMax_imputado_Razon_Normal_{i + 1}.csv'
    csv_path_imputado = os.path.join(output_directory_imputados, csv_filename_imputado)
    df_imputado.to_csv(csv_path_imputado, index=False)

print("Imputación realizada usando la técnica de la razón normal")

Imputación realizada usando la técnica de la razón normal


In [6]:
from sklearn.impute import KNNImputer

# Cargar los 15 dataframes con valores faltantes
dataframes_con_vacios = [pd.read_csv(os.path.join('Resultados', f'TempMax_con_vacios_{i + 1}', f'TempMax_con_vacios_{i + 1}.csv')) for i in range(15)]

# Configurar el imputador KNN
imputer = KNNImputer(n_neighbors=3)  #Ajustar el número de vecinos

# Imputar valores faltantes en cada DataFrame
dataframes_imputados_knn = []

for df in dataframes_con_vacios:
    # Seleccionar las columnas necesarias para la imputación
    features = df.drop(columns=['ID_CIDERH'])
    # Imputar valores faltantes
    imputed_array = imputer.fit_transform(features)
    # Asignar los valores imputados a las columnas correspondientes
    df['TEMP_MAX_RBE'] = imputed_array[:, features.columns.get_loc('TEMP_MAX_RBE')]
    dataframes_imputados_knn.append(df)

# Guardar los 15 dataframes imputados con KNN
output_directory_imputados_knn = 'Resultados_Imputacion_KNN'
os.makedirs(output_directory_imputados_knn, exist_ok=True)

for i, df_imputado_knn in enumerate(dataframes_imputados_knn):
    csv_filename_imputado_knn = f'TempMax_imputado_KNN_{i + 1}.csv'
    csv_path_imputado_knn = os.path.join(output_directory_imputados_knn, csv_filename_imputado_knn)
    df_imputado_knn.to_csv(csv_path_imputado_knn, index=False)

print("Imputación realizada usando la técnica de KNN")

Imputación realizada usando la técnica de KNN


In [7]:
# Cargar los 15 dataframes con valores faltantes
dataframes_con_vacios = [pd.read_csv(os.path.join('Resultados', f'TempMax_con_vacios_{i + 1}', f'TempMax_con_vacios_{i + 1}.csv')) for i in range(15)]

# Crear un diccionario que contenga las estaciones vecinas para cada estación
estaciones_vecinas = {
    'M_023_A_II': ['M_017_A_II', 'M_024_A_II', 'M_027_A_II'],
}

# Configurar el imputador WKNNI
def calcular_pesos(distancias):
    with np.errstate(divide='ignore', invalid='ignore'):
        # Manejar casos donde las distancias son pequeñas
        distancias_ajustadas = np.maximum(distancias, 1e-6)
        pesos = 1 / distancias_ajustadas
        pesos[~np.isfinite(pesos)] = 0
    return pesos

imputer_wknni = KNNImputer(n_neighbors=3, weights=calcular_pesos)  # Ajusta el número de vecinos

# Imputar valores faltantes en cada DataFrame
dataframes_imputados_wknni = []

for df in dataframes_con_vacios:
    # Seleccionar las columnas necesarias para la imputación
    features = df.drop(columns=['ID_CIDERH'])
    # Imputar valores faltantes con pesos en función de la distancia
    imputed_array = imputer_wknni.fit_transform(features)
    # Asegurar que los valores imputados estén dentro del rango permitido
    df['TEMP_MAX_RBE'] = np.clip(imputed_array[:, features.columns.get_loc('TEMP_MAX_RBE')], -np.inf, np.inf)
    dataframes_imputados_wknni.append(df)

# Guardar los 15 dataframes imputados con WKNNI
output_directory_imputados_wknni = 'Resultados_Imputacion_WKNNI'
os.makedirs(output_directory_imputados_wknni, exist_ok=True)

for i, df_imputado_wknni in enumerate(dataframes_imputados_wknni):
    csv_filename_imputado_wknni = f'TempMax_imputado_WKNNI_{i + 1}.csv'
    csv_path_imputado_wknni = os.path.join(output_directory_imputados_wknni, csv_filename_imputado_wknni)
    df_imputado_wknni.to_csv(csv_path_imputado_wknni, index=False)

print("Imputación realizada usando la técnica de WKNNI")

Imputación realizada usando la técnica de WKNNI


In [8]:
# Cargar los 15 dataframes con valores faltantes
dataframes_con_vacios = [pd.read_csv(os.path.join('Resultados', f'TempMax_con_vacios_{i + 1}', f'TempMax_con_vacios_{i + 1}.csv')) for i in range(15)]

# Crear un diccionario que contenga las estaciones vecinas para cada estación
estaciones_vecinas = {
    'M_023_A_II': ['M_017_A_II', 'M_024_A_II', 'M_027_A_II'],
}

# Realizar la imputación con Hot-Deck para cada dataframe y guardar los resultados
for i, df in enumerate(dataframes_con_vacios):
    estacion_actual = 'M_023_A_II'
    estaciones_vecinas_actual = estaciones_vecinas.get(estacion_actual, [])
    estaciones_df = df[df['ID_CIDERH'].isin([estacion_actual] + estaciones_vecinas_actual)]

    # Verificar si hay NaN en las observaciones
    if estaciones_df['TEMP_MAX_RBE'].isnull().any():
        # Iterar sobre las filas con NaN y reemplazar con valores de observaciones cercanas
        for idx, row in estaciones_df[estaciones_df['TEMP_MAX_RBE'].isnull()].iterrows():
            # Filtrar el DataFrame para obtener observaciones de la estación actual y las vecinas en el mismo día
            observaciones_cercanas = estaciones_df[
                (estaciones_df['Anio'] == row['Anio']) &
                (estaciones_df['Mes'] == row['Mes']) &
                (estaciones_df['Dia'] == row['Dia'])
            ]

            if not observaciones_cercanas.empty:
                # Tomar el primer valor no nulo en las observaciones cercanas y reemplazar
                valor_imputado = observaciones_cercanas['TEMP_MAX_RBE'].dropna().iloc[0]
                df.loc[idx, 'TEMP_MAX_RBE'] = valor_imputado

# Guardar los dataframes actualizados en nuevos CSVs
output_directory_imputados = 'Resultados_Imputacion_Hot_Deck'
os.makedirs(output_directory_imputados, exist_ok=True)

for i, df_imputado in enumerate(dataframes_con_vacios):
    csv_filename_imputado = f'TempMax_imputado_Hot_Deck_{i + 1}.csv'
    csv_path_imputado = os.path.join(output_directory_imputados, csv_filename_imputado)
    df_imputado.to_csv(csv_path_imputado, index=False)

print("Imputación realizada usando la técnica de Hot-Deck")

Imputación realizada usando la técnica de Hot-Deck


In [9]:
from sklearn.impute import KNNImputer
from scipy.spatial.distance import cdist

# Cargar los 15 dataframes con valores faltantes
dataframes_con_vacios = [pd.read_csv(os.path.join('Resultados', f'TempMax_con_vacios_{i + 1}', f'TempMax_con_vacios_{i + 1}.csv')) for i in range(15)]

# Crear un diccionario que contenga las estaciones vecinas para cada estación
estaciones_vecinas = {
    'M_023_A_II': ['M_017_A_II', 'M_024_A_II', 'M_027_A_II'],
}

# Función para calcular los pesos IDW
def calcular_pesos_idw(distancias):
    with np.errstate(divide='ignore', invalid='ignore'):
        pesos = 1 / distancias**2
        pesos[~np.isfinite(pesos)] = 0
    return pesos

# Imputar valores faltantes utilizando IDW
dataframes_imputados_idw = []

for df in dataframes_con_vacios:
    # Seleccionar las columnas necesarias para la imputación
    features = df.drop(columns=['ID_CIDERH', 'TEMP_MAX_RBE'])

    # Encontrar índices de filas con valores faltantes
    idx_valores_faltantes = df['TEMP_MAX_RBE'].isnull()

    # Iterar sobre las filas con valores faltantes
    for idx, row in df[idx_valores_faltantes].iterrows():
        estacion_actual = row['ID_CIDERH']
        estaciones_vecinas_actual = estaciones_vecinas.get(estacion_actual, [])

        # Filtrar el DataFrame para obtener observaciones de la estación actual y las vecinas
        estaciones_df = df[df['ID_CIDERH'].isin([estacion_actual] + estaciones_vecinas_actual)]
        observaciones = estaciones_df['TEMP_MAX_RBE']

        # Calcular distancias y pesos IDW
        distancias = cdist(features.loc[estaciones_df.index], [features.loc[idx]])
        pesos_idw = calcular_pesos_idw(distancias)

        # Imputar el valor faltante utilizando la suma ponderada
        valor_imputado = np.sum(observaciones * pesos_idw.flatten()) / np.sum(pesos_idw)

        # Asignar el valor imputado a la fila correspondiente
        df.loc[idx, 'TEMP_MAX_RBE'] = valor_imputado

    dataframes_imputados_idw.append(df)

# Guardar los 15 dataframes imputados con IDW
output_directory_imputados_idw = 'Resultados_Imputacion_IDW'
os.makedirs(output_directory_imputados_idw, exist_ok=True)

for i, df_imputado_idw in enumerate(dataframes_imputados_idw):
    csv_filename_imputado_idw = f'TempMax_imputado_IDW_{i + 1}.csv'
    csv_path_imputado_idw = os.path.join(output_directory_imputados_idw, csv_filename_imputado_idw)
    df_imputado_idw.to_csv(csv_path_imputado_idw, index=False)

print("Imputación realizada usando la técnica de IDW")

Imputación realizada usando la técnica de IDW


In [10]:
# Directorio que contiene los DataFrames con valores faltantes
input_directory = 'Resultados/TempMax_con_Vacios_{}'

# Directorio para almacenar los DataFrames actualizados
output_directory = 'Resultados_Imputados_ID'
os.makedirs(output_directory, exist_ok=True)

# Cargar los datos de RATempMin.csv
df_ra_temp_min = pd.read_csv(r"C:\Users\hctrl\OneDrive\Tesis\Experimento\Temperatura Máxima\RATempMax.csv")

# Iterar sobre cada DataFrame con valores faltantes
for i in range(15):  # Iterar sobre los 15 días
    folder_path = input_directory.format(i + 1)
    csv_filename = f'TempMax_con_vacios_{i + 1}.csv'
    csv_path = os.path.join(folder_path, csv_filename)
    df_temp = pd.read_csv(csv_path)
    
    # Realizar el reemplazo de valores nulos iterativamente con los datos de reanálisis
    for index, row in df_temp.iterrows():
        if pd.isnull(row['TEMP_MAX_RBE']):  # Si el valor está vacío
            fecha = f"{row['Anio']}-{row['Mes']}-{row['Dia']}"
            ra_temp_min_row = df_ra_temp_min[(df_ra_temp_min['ID_CIDERH'] == row['ID_CIDERH']) & 
                                              (df_ra_temp_min['Anio'] == row['Anio']) & 
                                              (df_ra_temp_min['Mes'] == row['Mes']) & 
                                              (df_ra_temp_min['Dia'] == row['Dia'])]
            if not ra_temp_min_row.empty:
                df_temp.at[index, 'TEMP_MAX_RBE'] = ra_temp_min_row['TEMP_MAX_RBE'].values[0]

    # Guardar el DataFrame actualizado con los valores reemplazados
    output_csv_path = os.path.join(output_directory, f'TempMax_imputado_ID_{i + 1}.csv')
    df_temp.to_csv(output_csv_path, index=False)

    print(f"Iteración {i + 1}: Valores imputados utilizando datos de reanálisis")

print("Proceso completado")

Iteración 1: Valores imputados utilizando datos de reanálisis
Iteración 2: Valores imputados utilizando datos de reanálisis
Iteración 3: Valores imputados utilizando datos de reanálisis
Iteración 4: Valores imputados utilizando datos de reanálisis
Iteración 5: Valores imputados utilizando datos de reanálisis
Iteración 6: Valores imputados utilizando datos de reanálisis
Iteración 7: Valores imputados utilizando datos de reanálisis
Iteración 8: Valores imputados utilizando datos de reanálisis
Iteración 9: Valores imputados utilizando datos de reanálisis
Iteración 10: Valores imputados utilizando datos de reanálisis
Iteración 11: Valores imputados utilizando datos de reanálisis
Iteración 12: Valores imputados utilizando datos de reanálisis
Iteración 13: Valores imputados utilizando datos de reanálisis
Iteración 14: Valores imputados utilizando datos de reanálisis
Iteración 15: Valores imputados utilizando datos de reanálisis
Proceso completado
